In [1]:
import tensorflow as tf
from __future__ import print_function
import numpy as np

/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
class BatchSampler(object):
    '''
    A (very) simple wrapper to randomly sample batches without replacement.
    '''
    
    def __init__(self, data, targets, batch_size):
        self.num_points = data.shape[0]
        self.features = data.shape[1]
        self.data = data
        self.targets = targets
        self.batch_size = batch_size
        self.indices = np.arange(self.num_points)

    def random_batch_indices(self, m=None):
        if m is None:
            indices = np.random.choice(self.indices, self.batch_size, replace=False)
        else:
            indices = np.random.choice(self.indices, m, replace=False)
        return indices 

    def get_batch(self, m=None):
        '''
        Get a random batch without replacement from the dataset.
        If m is given the batch will be of size m. 
        Otherwise will default to the class initialized value.
        '''
        indices = self.random_batch_indices(m)
        X_batch = np.take(self.data, indices, 0)
        y_batch = self.targets[indices]
        return X_batch, y_batch  


In [3]:
def loadBinData():
# import binary NOTMIST data set
    with np.load("notMNIST.npz") as data :
        Data, Target = data ["images"], data["labels"]
        posClass = 2
        negClass = 9
        dataIndx = (Target==posClass) + (Target==negClass)
        Data = Data[dataIndx]/255.
        Target = Target[dataIndx].reshape(-1, 1)
        Target[Target==posClass] = 1
        Target[Target==negClass] = 0
        np.random.seed(521)
        randIndx = np.arange(len(Data))
        np.random.shuffle(randIndx)
        Data, Target = Data[randIndx], Target[randIndx]
        trainData, trainTarget = Data[:3500], Target[:3500]
        validData, validTarget = Data[3500:3600], Target[3500:3600]
        testData, testTarget = Data[3600:], Target[3600:]
        
    print("Data binary class Loaded")
    print("-------------------------------")
    return trainData, trainTarget,validData, validTarget,\
        testData, testTarget

def arrFlatten(arr):
    
    print(arr.shape)

    dataDim1, dum1, dum2 = arr.shape
    arr = np.reshape(arr, [ dataDim1 ,784 ])
    return arr



def linearMSE(y_hat, target):
    '''
    TODO: the MSE calculation
    '''
    #print("y_hat",  y_hat)
    
    se_mat = tf.square(tf.subtract(y_hat, target))
    #print("msemst", mse_mat)
    mse_mat = tf.reduce_mean(se_mat)
    loss = tf.reduce_mean(mse_mat)
    loss = tf.div(loss, tf.constant(2.0))
    
    #loss = tf.div(tf.reduce_mean(tf.reduce_mean(tf.square(y_hat - target), 
                                #                reduction_indices=1, 
                                #                name='squared_error'), 
                                #  name='mean_squared_error'), tf.constant(2.0))
    return loss

def linearBuildGraph(_regLambda, _learningRate):
    '''
    Input: _data is x in the equation, dim by 784 flattened tensor
       _target is y in the equaion
       _regLambda is the wegithed decay coeff
       _learningRate is the epsilon
    '''

    #declare using a placeholder, feed in _data and _target to x ,y 
    #x_dim, dum1 =_data.get_shape().as_list()
    X = tf.placeholder(tf.float32, shape=[None, 784], name='dataX')
    # W initialize to a gaussian distr, honestly anything would work
    W = tf.Variable(tf.truncated_normal(shape=[784, 1], stddev=0.1), name='weights')
    W = tf.cast(W, dtype=tf.float32)
    b = tf.Variable(0.0, name='biases')
    y_target = tf.placeholder(tf.float32, shape=[None, 1], name='targetY')
    
    #compute the current y_hat
    y_hat =  tf.matmul(X, W) + b
    #compute the current loss
    mseCurr = linearMSE(y_hat, y_target)
    print("mseerRor 1", mseCurr)

    #compute the decay/regularization term
    regTerm = tf.multiply(_regLambda, tf.reduce_mean(tf.square(W)))
    print("regTerm", regTerm)
    mseCurr = tf.add(mseCurr, regTerm)
    print("mseerRor", mseCurr)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate = _learningRate)
    
    train = optimizer.minimize(loss=mseCurr)

    return W, b, mseCurr, y_hat, X, y_target, train
    

def runLinearGraph(trainData, trainTarget ):
    
    '''
    Input: _data,
           _target,
           _numIters
    Output:
           required accuracy/epoch plots
           
    '''
    
    regLambda = 0.0
    learningRateArr = [0.005, 0.001, 0.0001]
    numIter = 20000
    numEpoch =int(np.ceil(20000/7))
    batchSize = 500
    
    
    for learningRate in learningRateArr:
        tf.reset_default_graph()
        W, b, mseError, y_hat, X, y_target, train = linearBuildGraph(regLambda, learningRate)
    

        init = tf.global_variables_initializer()
        sess = tf.InteractiveSession()
        sess.run(init)
        initialW = sess.run(W)  
        initialb = sess.run(b)            
        #training model and iter through batches
        
        for i in range(numEpoch):
            trainBatchSampler = BatchSampler(trainData, trainTarget, batchSize)
            dataBatch, targetBatch = trainBatchSampler.get_batch()
            #dataBatch = tf.stack(dataBatch)
            #targetBatch = tf.stack(targetBatch)
            print("databatch", dataBatch.shape)
            print("targetBatch", targetBatch.shape)
            print("feed ", i, "times")
            currentW, currentb, errTrain, y_predict, trainModel = sess.run([W, b, mseError, y_hat, train], feed_dict={X: dataBatch, y_target: targetBatch})
   

    
        print(errTrain)
        print("done")
    

if __name__ == '__main__':  
        

    #trainTarget = arrFlatten(trainTarget)
    trainData, trainTarget, validData, validTarget,testData, testTarget = loadBinData()
    #data is (3500, 28, 28)
    #the label [1] or [0] is stored in target
    trainData = arrFlatten(trainData)
    runLinearGraph(trainData, trainTarget)
    

Data binary class Loaded
-------------------------------
(3500, 28, 28)
mseerRor 1 Tensor("div:0", shape=(), dtype=float32)
regTerm Tensor("Mul:0", shape=(), dtype=float32)
mseerRor Tensor("Add:0", shape=(), dtype=float32)
databatch (500, 784)
targetBatch (500, 1)
feed  0 times
databatch (500, 784)
targetBatch (500, 1)
feed  1 times
databatch (500, 784)
targetBatch (500, 1)
feed  2 times
databatch (500, 784)
targetBatch (500, 1)
feed  3 times
databatch (500, 784)
targetBatch (500, 1)
feed  4 times
databatch (500, 784)
targetBatch (500, 1)
feed  5 times
databatch (500, 784)
targetBatch (500, 1)
feed  6 times
databatch (500, 784)
targetBatch (500, 1)
feed  7 times
databatch (500, 784)
targetBatch (500, 1)
feed  8 times
databatch (500, 784)
targetBatch (500, 1)
feed  9 times
databatch (500, 784)
targetBatch (500, 1)
feed  10 times
databatch (500, 784)
targetBatch (500, 1)
feed  11 times
databatch (500, 784)
targetBatch (500, 1)
feed  12 times
databatch (500, 784)
targetBatch (500, 1)
feed

databatch (500, 784)
targetBatch (500, 1)
feed  241 times
databatch (500, 784)
targetBatch (500, 1)
feed  242 times
databatch (500, 784)
targetBatch (500, 1)
feed  243 times
databatch (500, 784)
targetBatch (500, 1)
feed  244 times
databatch (500, 784)
targetBatch (500, 1)
feed  245 times
databatch (500, 784)
targetBatch (500, 1)
feed  246 times
databatch (500, 784)
targetBatch (500, 1)
feed  247 times
databatch (500, 784)
targetBatch (500, 1)
feed  248 times
databatch (500, 784)
targetBatch (500, 1)
feed  249 times
databatch (500, 784)
targetBatch (500, 1)
feed  250 times
databatch (500, 784)
targetBatch (500, 1)
feed  251 times
databatch (500, 784)
targetBatch (500, 1)
feed  252 times
databatch (500, 784)
targetBatch (500, 1)
feed  253 times
databatch (500, 784)
targetBatch (500, 1)
feed  254 times
databatch (500, 784)
targetBatch (500, 1)
feed  255 times
databatch (500, 784)
targetBatch (500, 1)
feed  256 times
databatch (500, 784)
targetBatch (500, 1)
feed  257 times
databatch (500

databatch (500, 784)
targetBatch (500, 1)
feed  464 times
databatch (500, 784)
targetBatch (500, 1)
feed  465 times
databatch (500, 784)
targetBatch (500, 1)
feed  466 times
databatch (500, 784)
targetBatch (500, 1)
feed  467 times
databatch (500, 784)
targetBatch (500, 1)
feed  468 times
databatch (500, 784)
targetBatch (500, 1)
feed  469 times
databatch (500, 784)
targetBatch (500, 1)
feed  470 times
databatch (500, 784)
targetBatch (500, 1)
feed  471 times
databatch (500, 784)
targetBatch (500, 1)
feed  472 times
databatch (500, 784)
targetBatch (500, 1)
feed  473 times
databatch (500, 784)
targetBatch (500, 1)
feed  474 times
databatch (500, 784)
targetBatch (500, 1)
feed  475 times
databatch (500, 784)
targetBatch (500, 1)
feed  476 times
databatch (500, 784)
targetBatch (500, 1)
feed  477 times
databatch (500, 784)
targetBatch (500, 1)
feed  478 times
databatch (500, 784)
targetBatch (500, 1)
feed  479 times
databatch (500, 784)
targetBatch (500, 1)
feed  480 times
databatch (500

databatch (500, 784)
targetBatch (500, 1)
feed  682 times
databatch (500, 784)
targetBatch (500, 1)
feed  683 times
databatch (500, 784)
targetBatch (500, 1)
feed  684 times
databatch (500, 784)
targetBatch (500, 1)
feed  685 times
databatch (500, 784)
targetBatch (500, 1)
feed  686 times
databatch (500, 784)
targetBatch (500, 1)
feed  687 times
databatch (500, 784)
targetBatch (500, 1)
feed  688 times
databatch (500, 784)
targetBatch (500, 1)
feed  689 times
databatch (500, 784)
targetBatch (500, 1)
feed  690 times
databatch (500, 784)
targetBatch (500, 1)
feed  691 times
databatch (500, 784)
targetBatch (500, 1)
feed  692 times
databatch (500, 784)
targetBatch (500, 1)
feed  693 times
databatch (500, 784)
targetBatch (500, 1)
feed  694 times
databatch (500, 784)
targetBatch (500, 1)
feed  695 times
databatch (500, 784)
targetBatch (500, 1)
feed  696 times
databatch (500, 784)
targetBatch (500, 1)
feed  697 times
databatch (500, 784)
targetBatch (500, 1)
feed  698 times
databatch (500

databatch (500, 784)
targetBatch (500, 1)
feed  885 times
databatch (500, 784)
targetBatch (500, 1)
feed  886 times
databatch (500, 784)
targetBatch (500, 1)
feed  887 times
databatch (500, 784)
targetBatch (500, 1)
feed  888 times
databatch (500, 784)
targetBatch (500, 1)
feed  889 times
databatch (500, 784)
targetBatch (500, 1)
feed  890 times
databatch (500, 784)
targetBatch (500, 1)
feed  891 times
databatch (500, 784)
targetBatch (500, 1)
feed  892 times
databatch (500, 784)
targetBatch (500, 1)
feed  893 times
databatch (500, 784)
targetBatch (500, 1)
feed  894 times
databatch (500, 784)
targetBatch (500, 1)
feed  895 times
databatch (500, 784)
targetBatch (500, 1)
feed  896 times
databatch (500, 784)
targetBatch (500, 1)
feed  897 times
databatch (500, 784)
targetBatch (500, 1)
feed  898 times
databatch (500, 784)
targetBatch (500, 1)
feed  899 times
databatch (500, 784)
targetBatch (500, 1)
feed  900 times
databatch (500, 784)
targetBatch (500, 1)
feed  901 times
databatch (500

...

databatch (500, 784)
targetBatch (500, 1)
feed  1715 times
databatch (500, 784)
targetBatch (500, 1)
feed  1716 times
databatch (500, 784)
targetBatch (500, 1)
feed  1717 times
databatch (500, 784)
targetBatch (500, 1)
feed  1718 times
databatch (500, 784)
targetBatch (500, 1)
feed  1719 times
databatch (500, 784)
targetBatch (500, 1)
feed  1720 times
databatch (500, 784)
targetBatch (500, 1)
feed  1721 times
databatch (500, 784)
targetBatch (500, 1)
feed  1722 times
databatch (500, 784)
targetBatch (500, 1)
feed  1723 times
databatch (500, 784)
targetBatch (500, 1)
feed  1724 times
databatch (500, 784)
targetBatch (500, 1)
feed  1725 times
databatch (500, 784)
targetBatch (500, 1)
feed  1726 times
databatch (500, 784)
targetBatch (500, 1)
feed  1727 times
databatch (500, 784)
targetBatch (500, 1)
feed  1728 times
databatch (500, 784)
targetBatch (500, 1)
feed  1729 times
databatch (500, 784)
targetBatch (500, 1)
feed  1730 times
databatch (500, 784)
targetBatch (500, 1)
feed  1731 tim

databatch (500, 784)
targetBatch (500, 1)
feed  1952 times
databatch (500, 784)
targetBatch (500, 1)
feed  1953 times
databatch (500, 784)
targetBatch (500, 1)
feed  1954 times
databatch (500, 784)
targetBatch (500, 1)
feed  1955 times
databatch (500, 784)
targetBatch (500, 1)
feed  1956 times
databatch (500, 784)
targetBatch (500, 1)
feed  1957 times
databatch (500, 784)
targetBatch (500, 1)
feed  1958 times
databatch (500, 784)
targetBatch (500, 1)
feed  1959 times
databatch (500, 784)
targetBatch (500, 1)
feed  1960 times
databatch (500, 784)
targetBatch (500, 1)
feed  1961 times
databatch (500, 784)
targetBatch (500, 1)
feed  1962 times
databatch (500, 784)
targetBatch (500, 1)
feed  1963 times
databatch (500, 784)
targetBatch (500, 1)
feed  1964 times
databatch (500, 784)
targetBatch (500, 1)
feed  1965 times
databatch (500, 784)
targetBatch (500, 1)
feed  1966 times
databatch (500, 784)
targetBatch (500, 1)
feed  1967 times
databatch (500, 784)
targetBatch (500, 1)
feed  1968 tim

databatch (500, 784)
targetBatch (500, 1)
feed  2187 times
databatch (500, 784)
targetBatch (500, 1)
feed  2188 times
databatch (500, 784)
targetBatch (500, 1)
feed  2189 times
databatch (500, 784)
targetBatch (500, 1)
feed  2190 times
databatch (500, 784)
targetBatch (500, 1)
feed  2191 times
databatch (500, 784)
targetBatch (500, 1)
feed  2192 times
databatch (500, 784)
targetBatch (500, 1)
feed  2193 times
databatch (500, 784)
targetBatch (500, 1)
feed  2194 times
databatch (500, 784)
targetBatch (500, 1)
feed  2195 times
databatch (500, 784)
targetBatch (500, 1)
feed  2196 times
databatch (500, 784)
targetBatch (500, 1)
feed  2197 times
databatch (500, 784)
targetBatch (500, 1)
feed  2198 times
databatch (500, 784)
targetBatch (500, 1)
feed  2199 times
databatch (500, 784)
targetBatch (500, 1)
feed  2200 times
databatch (500, 784)
targetBatch (500, 1)
feed  2201 times
databatch (500, 784)
targetBatch (500, 1)
feed  2202 times
databatch (500, 784)
targetBatch (500, 1)
feed  2203 tim

databatch (500, 784)
targetBatch (500, 1)
feed  2425 times
databatch (500, 784)
targetBatch (500, 1)
feed  2426 times
databatch (500, 784)
targetBatch (500, 1)
feed  2427 times
databatch (500, 784)
targetBatch (500, 1)
feed  2428 times
databatch (500, 784)
targetBatch (500, 1)
feed  2429 times
databatch (500, 784)
targetBatch (500, 1)
feed  2430 times
databatch (500, 784)
targetBatch (500, 1)
feed  2431 times
databatch (500, 784)
targetBatch (500, 1)
feed  2432 times
databatch (500, 784)
targetBatch (500, 1)
feed  2433 times
databatch (500, 784)
targetBatch (500, 1)
feed  2434 times
databatch (500, 784)
targetBatch (500, 1)
feed  2435 times
databatch (500, 784)
targetBatch (500, 1)
feed  2436 times
databatch (500, 784)
targetBatch (500, 1)
feed  2437 times
databatch (500, 784)
targetBatch (500, 1)
feed  2438 times
databatch (500, 784)
targetBatch (500, 1)
feed  2439 times
databatch (500, 784)
targetBatch (500, 1)
feed  2440 times
databatch (500, 784)
targetBatch (500, 1)
feed  2441 tim

databatch (500, 784)
targetBatch (500, 1)
feed  2656 times
databatch (500, 784)
targetBatch (500, 1)
feed  2657 times
databatch (500, 784)
targetBatch (500, 1)
feed  2658 times
databatch (500, 784)
targetBatch (500, 1)
feed  2659 times
databatch (500, 784)
targetBatch (500, 1)
feed  2660 times
databatch (500, 784)
targetBatch (500, 1)
feed  2661 times
databatch (500, 784)
targetBatch (500, 1)
feed  2662 times
databatch (500, 784)
targetBatch (500, 1)
feed  2663 times
databatch (500, 784)
targetBatch (500, 1)
feed  2664 times
databatch (500, 784)
targetBatch (500, 1)
feed  2665 times
databatch (500, 784)
targetBatch (500, 1)
feed  2666 times
databatch (500, 784)
targetBatch (500, 1)
feed  2667 times
databatch (500, 784)
targetBatch (500, 1)
feed  2668 times
databatch (500, 784)
targetBatch (500, 1)
feed  2669 times
databatch (500, 784)
targetBatch (500, 1)
feed  2670 times
databatch (500, 784)
targetBatch (500, 1)
feed  2671 times
databatch (500, 784)
targetBatch (500, 1)
feed  2672 tim